In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,ShuffleSplit,cross_val_score
from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor

In [4]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[['League','Division','NewLeague']])
y = df["Salary"]
x_ = df.drop(['Salary','League','Division','NewLeague'],axis=1).astype("float64")
x = pd.concat([x_,dms[['League_N','Division_W','NewLeague_N']]],axis=1)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=42)

In [5]:
knn_model = KNeighborsRegressor().fit(x_train,y_train)

In [6]:
knn_model

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform')

In [8]:
#Tahmin

y_pred = knn_model.predict(x_test)

In [9]:
np.sqrt(mean_squared_error(y_test,y_pred))

426.6570764525201

In [10]:
RMSE = []

for k in range(10):
    k = k+1
    knn_model = KNeighborsRegressor(n_neighbors = k).fit(x_train,y_train)
    y_pred = knn_model.predict(x_train)
    rmse = np.sqrt(mean_squared_error(y_train,y_pred))
    RMSE.append(rmse)
    print("k = ",k,"için RMSE değeri: ",rmse)

k =  1 için RMSE değeri:  0.0
k =  2 için RMSE değeri:  179.52761335480352
k =  3 için RMSE değeri:  205.20157172291863
k =  4 için RMSE değeri:  220.5139794876305
k =  5 için RMSE değeri:  239.64671325413764
k =  6 için RMSE değeri:  243.5904190007242
k =  7 için RMSE değeri:  258.1478781634636
k =  8 için RMSE değeri:  266.05374203349805
k =  9 için RMSE değeri:  269.73782093553376
k =  10 için RMSE değeri:  271.2798300436963


In [11]:
#Model Tuning

from sklearn.model_selection import GridSearchCV

In [14]:
knn_params = {"n_neighbors":np.arange(1,30,1)}

In [15]:
knn = KNeighborsRegressor()

In [16]:
knn_cv_model = GridSearchCV(knn,knn_params,cv=10)

In [17]:
knn_cv_model.fit(x_train,y_train)

C:\Users\Kerem\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [18]:
knn_cv_model.best_params_["n_neighbors"]

8

In [20]:
RMSE = []
RMSE_CV = []

for k in range(10):
    k = k+1
    knn_model = KNeighborsRegressor(n_neighbors = k).fit(x_train,y_train)
    y_pred = knn_model.predict(x_train)
    rmse = np.sqrt(mean_squared_error(y_train,y_pred))
    rmse_cv = np.sqrt(-1*cross_val_score(knn_model,x_train,y_train,cv=10,
                                         scoring = "neg_mean_squared_error").mean())
    RMSE.append(rmse)
    RMSE_CV.append(rmse_cv)
    print("k =",k," için RMSE değeri: ",rmse," RMSE_CV değeri: ",rmse_cv)

k = 1  için RMSE değeri:  0.0  RMSE_CV değeri:  325.39475147063825
k = 2  için RMSE değeri:  179.52761335480352  RMSE_CV değeri:  293.24000183333817
k = 3  için RMSE değeri:  205.20157172291863  RMSE_CV değeri:  283.7486667487823
k = 4  için RMSE değeri:  220.5139794876305  RMSE_CV değeri:  286.3240222024089
k = 5  için RMSE değeri:  239.64671325413764  RMSE_CV değeri:  290.0705466132226
k = 6  için RMSE değeri:  243.5904190007242  RMSE_CV değeri:  298.1263115575851
k = 7  için RMSE değeri:  258.1478781634636  RMSE_CV değeri:  294.77070479194987
k = 8  için RMSE değeri:  266.05374203349805  RMSE_CV değeri:  291.98672028891235
k = 9  için RMSE değeri:  269.73782093553376  RMSE_CV değeri:  295.7162739573105
k = 10  için RMSE değeri:  271.2798300436963  RMSE_CV değeri:  301.31047022701154


In [21]:
knn_tuned = KNeighborsRegressor(n_neighbors=knn_cv_model.best_params_["n_neighbors"])

In [23]:
knn_tuned.fit(x_train,y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=8, p=2,
          weights='uniform')

In [24]:
np.sqrt(mean_squared_error(y_test,knn_tuned.predict(x_test)))

413.7094731463598